In [50]:
import pandas as pd

import numpy as np
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn import datasets, linear_model, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
import warnings
warnings.filterwarnings('ignore')

In [51]:
satellite = pd.read_csv(r'Satellite.csv',sep=';')

In [52]:
satellite.head()

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
0,92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
1,84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
2,84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
3,80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
4,84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil


In [53]:
X = satellite.drop('classes',axis=1)
y = satellite['classes']

In [54]:
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=24,test_size=0.3)

#### Linear Discriminant Analysis

In [55]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train,y_train)
y_pred = lda.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8353184878301398


#### Quadratic Discriminant Analysis

In [56]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train,y_train)
y_pred = qda.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8503366131538063


#### KNN + LDA

In [71]:
knn = KNeighborsClassifier(n_neighbors=3)

In [72]:
lda = LinearDiscriminantAnalysis()
pipe = Pipeline([('LDA',lda),('KNN',knn)])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8700155359917141


In [73]:
params = {'KNN__n_neighbors':np.arange(1,5)}

In [74]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [75]:
gcv = GridSearchCV(pipe,param_grid=params,cv=kfold)

In [76]:
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis()),
                                       ('KNN',
                                        KNeighborsClassifier(n_neighbors=3))]),
             param_grid={'KNN__n_neighbors': array([1, 2, 3, 4])})

In [77]:
gcv.best_params_

{'KNN__n_neighbors': 4}

In [78]:
gcv.best_score_

0.8749028749028749